In [1]:
import pandas as pd
import numpy as np
import scipy as scipy
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import wilcoxon
from scipy.stats import chi2_contingency
from scipy.stats import chisquare

Nama Anggota Kelompok:

1. Caren Ryana (23101910025)
2. Enrico Wijaya (23101910058)
3. Jeth Cuthbert Sean (23101910022)
4. Steffi Graciela (23101910067)

In [2]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=d7601dc494f1e1aba9066bcb9ad58e7d664da8a2626df5cf437273371fe12556
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [3]:
df = pd.read_excel('Groceries_labeled.xlsx')

In [4]:
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week,Category
0,1249,2014-01-01,citrus fruit,2014,1,1,2,Fruits/Vegetables
1,1249,2014-01-01,coffee,2014,1,1,2,Beverages
2,1381,2014-01-01,curd,2014,1,1,2,Dairy
3,1381,2014-01-01,soda,2014,1,1,2,Beverages
4,1440,2014-01-01,other vegetables,2014,1,1,2,Fruits/Vegetables


In [5]:
df['Category'].unique()

array(['Fruits/Vegetables', 'Beverages', 'Dairy', 'Snacks',
       'Frozen_Foods', 'Meat', 'Household&Cleaning Supplies', 'Deli',
       'Plants', 'Bread&Bakery', 'Condiments&Spices', 'Others',
       'Canned_Goods', 'Pet_Care', 'Baking', 'Pasta&Rice&Cereal',
       'Health&Personal_Care'], dtype=object)

# Data Understanding

# Data Analysis

## Churn Analysis

In [6]:
item = df[['Member_number','year','itemDescription']].groupby(by=['Member_number','year'],as_index=False).count()

In [7]:
item.rename(columns={'itemDescription':'items'},inplace=True)

In [8]:
item['items_2014']=item[item['year']==2014]['items']
item['items_2015']=item[item['year']==2015]['items']

In [9]:
item=item[['Member_number','items','items_2014','items_2015']].groupby(by=['Member_number'],as_index=False).sum()

In [10]:
item.head()

,Member_number,items,items_2014,items_2015
0,1000,13,3.0,10.0
1,1001,12,5.0,7.0
2,1002,8,4.0,4.0
3,1003,8,6.0,2.0
4,1004,21,19.0,2.0


In [11]:
stats.ttest_rel(item['items_2014'],item['items_2015'])

Ttest_relResult(statistic=-6.3990119286601725, pvalue=1.749171829328491e-10)

In [12]:
trans = df[['Member_number','year','Date']]
trans = trans.groupby(by=['Member_number','year','Date'],as_index=False).count()
trans['transactions']=1
trans = trans[['Member_number','year','transactions']].groupby(by=['Member_number','year'],as_index=False).sum()

In [13]:
trans['transactions_2014']=trans[trans['year']==2014]['transactions']
trans['transactions_2015']=trans[trans['year']==2015]['transactions']

In [14]:
trans=trans[['Member_number','transactions','transactions_2014','transactions_2015']].groupby(by=['Member_number'],as_index=False).sum()

In [15]:
trans.head()

,Member_number,transactions,transactions_2014,transactions_2015
0,1000,5,1.0,4.0
1,1001,5,2.0,3.0
2,1002,4,2.0,2.0
3,1003,4,3.0,1.0
4,1004,8,7.0,1.0


In [16]:
stats.ttest_rel(trans['transactions_2014'],trans['transactions_2015'])

Ttest_relResult(statistic=8.163112501584244, pvalue=4.3637639958439126e-16)

In [17]:
ans = trans.merge(item, on='Member_number')
ans

,Member_number,transactions,transactions_2014,transactions_2015,items,items_2014,items_2015
0,1000,5,1.0,4.0,13,3.0,10.0
1,1001,5,2.0,3.0,12,5.0,7.0
2,1002,4,2.0,2.0,8,4.0,4.0
3,1003,4,3.0,1.0,8,6.0,2.0
4,1004,8,7.0,1.0,21,19.0,2.0
...,...,...,...,...,...,...,...
3893,4996,3,0.0,3.0,10,0.0,10.0
3894,4997,2,1.0,1.0,6,2.0,4.0
3895,4998,1,0.0,1.0,2,0.0,2.0
3896,4999,6,2.0,4.0,16,4.0,12.0


In [18]:
ans['churn']=''
for i in range(len(ans)):
  if (ans['items_2015'][i]==0) & (ans['items_2014'][i]>0):
    ans['churn'][i]='churn'
  elif (ans['items_2015'][i]>0) & (ans['items_2014'][i]>0):
    ans['churn'][i]='not_churn'
  elif (ans['items_2015'][i]>0) & (ans['items_2014'][i]==0):
    ans['churn'][i]='new_cust'

<ipython-input-18-974d9361aa8b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['churn'][i]='not_churn'
<ipython-input-18-974d9361aa8b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['churn'][i]='churn'
<ipython-input-18-974d9361aa8b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['churn'][i]='new_cust'


In [19]:
ans.head()

,Member_number,transactions,transactions_2014,transactions_2015,items,items_2014,items_2015,churn
0,1000,5,1.0,4.0,13,3.0,10.0,not_churn
1,1001,5,2.0,3.0,12,5.0,7.0,not_churn
2,1002,4,2.0,2.0,8,4.0,4.0,not_churn
3,1003,4,3.0,1.0,8,6.0,2.0,not_churn
4,1004,8,7.0,1.0,21,19.0,2.0,not_churn


In [20]:
ans['change_in_trans']=ans['items_2015']-ans['items_2014']
ans['items_movement']=''
for i in range(len(ans)):
  if (ans['churn'][i] =='not_churn') & (ans['change_in_trans'][i]<0):
    ans['items_movement'][i]='decrease'
  elif (ans['churn'][i] =='not_churn') & (ans['change_in_trans'][i]>0):
    ans['items_movement'][i]='increase'
  elif (ans['churn'][i] =='not_churn') & (ans['change_in_trans'][i]==0):
    ans['items_movement'][i]='remain'
  else:
    ans['items_movement'][i]='none'

<ipython-input-20-f42a1ffff757>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['items_movement'][i]='increase'
<ipython-input-20-f42a1ffff757>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['items_movement'][i]='remain'
<ipython-input-20-f42a1ffff757>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['items_movement'][i]='decrease'
<ipython-input-20-f42a1ffff757>:11: SettingWithCopyWarning: 
A value is trying to be set o

In [21]:
ans.head()

,Member_number,transactions,transactions_2014,transactions_2015,items,items_2014,items_2015,churn,change_in_trans,items_movement
0,1000,5,1.0,4.0,13,3.0,10.0,not_churn,7.0,increase
1,1001,5,2.0,3.0,12,5.0,7.0,not_churn,2.0,increase
2,1002,4,2.0,2.0,8,4.0,4.0,not_churn,0.0,remain
3,1003,4,3.0,1.0,8,6.0,2.0,not_churn,-4.0,decrease
4,1004,8,7.0,1.0,21,19.0,2.0,not_churn,-17.0,decrease


In [22]:
print("churn rate tahun 2014 -> 2015 = ",round(len(ans[ans['churn']=='churn'])/(len(ans[ans['churn']=='churn'])+len(ans[ans['churn']=='not_churn']))*100,2),'%')

churn rate tahun 2014 -> 2015 =  16.96 %


In [23]:
print("new customer growth rate tahun 2014 -> 2015 = ",round(len(ans[ans['churn']=='new_cust'])/(len(ans[ans['churn']=='churn'])+len(ans[ans['churn']=='not_churn']))*100,2),'%')

new customer growth rate tahun 2014 -> 2015 =  13.22 %


In [24]:
ans.to_excel('churn_analysis_grocery.xlsx')

## Before & After Analysis (Movement)

In [25]:
BAA = df.copy()
BAA['month'] =BAA['month'].replace({1:'january',2:'february',3:'march',4:'april',5:'may',6:'june',7:'july',8:'august',9:'september',10:'october',11:'november',12:'december'})
BAA['day_of_week']=BAA['day_of_week'].replace({0:'monday',1:'tuesday',2:'wednesday',3:'thursday',4:'friday',5:'saturday',6:'sunday'})

In [26]:
#
BAA['count']=1
month_ans_BA=BAA[['month','year','count']].groupby(by=['month','year'],as_index=False).sum()
day_ans_BA=BAA[['day_of_week','year','count']].groupby(by=['day_of_week','year'],as_index=False).sum()

In [27]:
testdiff_month = pd.pivot_table(month_ans_BA, values='count', index=['month'], columns=['year'], aggfunc=np.sum)
testdiff_month

year,2014,2015
month,,
april,1506,1666
august,1535,1963
december,1520,1554
february,1547,1485
january,1504,1829
july,1623,1645
june,1525,1791
march,1491,1792
may,1625,1710


In [28]:
print('hasil p value dari wilcoxon test wilcoxon = ',wilcoxon(testdiff_month[2015]-testdiff_month[2014]).pvalue)

hasil p value dari wilcoxon test wilcoxon =  0.00244140625


Reject null hypothesis, mean 2014 tidak sama dengan 2015

In [29]:
testdiff_day = pd.pivot_table(day_ans_BA, values='count', index=['day_of_week'], columns=['year'], aggfunc=np.sum)
testdiff_day

year,2014,2015
day_of_week,,
friday,2621,2883
monday,2548,2751
saturday,2560,2837
sunday,2624,3000
thursday,2798,2956
tuesday,2594,2930
wednesday,2532,3131


In [30]:
print('hasil p value dari wilcoxon test wilcoxon = ',wilcoxon(testdiff_day[2015]-testdiff_day[2014]).pvalue)

hasil p value dari wilcoxon test wilcoxon =  0.015625


In [31]:
month_ans_group =BAA[['month','itemDescription','count']].groupby(by=['month','itemDescription'],as_index=False).sum()
day_ans_group =BAA[['day_of_week','itemDescription','count']].groupby(by=['day_of_week','itemDescription'],as_index=False).sum()

In [32]:
testdiff_month_within = pd.pivot_table(month_ans_group, values='count', index=['itemDescription'], columns=['month'], aggfunc=np.sum)
testdiff_month_within =testdiff_month_within.fillna(0)
testdiff_month_within 

month,april,august,december,february,january,july,june,march,may,november,october,september
itemDescription,,,,,,,,,,,,
Instant food products,4.0,6.0,7.0,11.0,5.0,7.0,1.0,5.0,5.0,2.0,7.0,0.0
UHT-milk,32.0,33.0,26.0,22.0,20.0,33.0,26.0,27.0,36.0,25.0,23.0,20.0
abrasive cleaner,1.0,2.0,1.0,2.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
artif. sweetener,3.0,3.0,1.0,4.0,3.0,2.0,3.0,4.0,2.0,0.0,1.0,3.0
baby cosmetics,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
white bread,27.0,28.0,28.0,30.0,34.0,31.0,45.0,18.0,26.0,37.0,36.0,22.0
white wine,17.0,20.0,10.0,21.0,14.0,18.0,11.0,15.0,20.0,9.0,10.0,11.0
whole milk,218.0,235.0,167.0,173.0,219.0,222.0,205.0,216.0,218.0,230.0,199.0,200.0


In [33]:
stats.kruskal(testdiff_month_within['january'],testdiff_month_within['february'],testdiff_month_within['march'],testdiff_month_within['april'],testdiff_month_within['may'],testdiff_month_within['june'],testdiff_month_within['july'],
              testdiff_month_within['august'],testdiff_month_within['september'],testdiff_month_within['october'],testdiff_month_within['november'],testdiff_month_within['december'])

KruskalResult(statistic=2.02761443825316, pvalue=0.9983964720744573)

In [34]:
stats.chi2_contingency(testdiff_month_within)[1]

0.040263498821260284

In [35]:
testdiff_day_within = pd.pivot_table(day_ans_group, values='count', index=['itemDescription'], columns=['day_of_week'], aggfunc=np.sum)
testdiff_day_within =testdiff_day_within.fillna(0)
testdiff_day_within 

day_of_week,friday,monday,saturday,sunday,thursday,tuesday,wednesday
itemDescription,,,,,,,
Instant food products,9.0,7.0,8.0,8.0,7.0,5.0,16.0
UHT-milk,37.0,47.0,41.0,51.0,46.0,46.0,55.0
abrasive cleaner,2.0,2.0,2.0,4.0,4.0,2.0,6.0
artif. sweetener,4.0,4.0,5.0,2.0,6.0,5.0,3.0
baby cosmetics,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
white bread,60.0,47.0,62.0,44.0,50.0,52.0,47.0
white wine,31.0,23.0,19.0,22.0,28.0,26.0,27.0
whole milk,337.0,349.0,347.0,378.0,359.0,343.0,389.0


In [36]:
stats.kruskal(testdiff_day_within['monday'],testdiff_day_within['tuesday'],testdiff_day_within['wednesday'],
              testdiff_day_within['thursday'],testdiff_day_within['friday'],testdiff_day_within['saturday'],testdiff_day_within['sunday'])

KruskalResult(statistic=0.4676865709135297, pvalue=0.9982097779653493)

In [37]:
stats.chi2_contingency(testdiff_day_within)[1]

0.9751440208775412

## Persona Analysis

In [38]:
df1=df[['Member_number','Date','itemDescription','Category','year']]
df2=ans[['Member_number','churn','items_movement']]
df_labeled=df1.merge(df2,on='Member_number')

In [39]:
df_labeled.head()

,Member_number,Date,itemDescription,Category,year,churn,items_movement
0,1249,2014-01-01,citrus fruit,Fruits/Vegetables,2014,churn,none
1,1249,2014-01-01,coffee,Beverages,2014,churn,none
2,1249,2014-11-10,soda,Beverages,2014,churn,none
3,1249,2014-11-10,frozen meals,Frozen_Foods,2014,churn,none
4,1249,2014-11-10,bottled beer,Beverages,2014,churn,none


In [40]:
persona = df_labeled.copy()
persona['persona']= persona['churn']+"_"+persona['items_movement']
persona['count']=1
persona =persona[persona['churn']!='churn'][['Category','persona','count']].groupby(by=['Category','persona'],as_index=False).sum()
persona = pd.pivot_table(persona, values='count', index=['Category'], columns=['persona'], aggfunc=np.sum)
persona = persona.fillna(0)
persona

persona,new_cust_none,not_churn_decrease,not_churn_increase,not_churn_remain
Category,,,,
Baking,11,51,84,16
Beverages,431,2025,2885,535
Bread&Bakery,248,1288,1789,304
Canned_Goods,20,118,147,28
Condiments&Spices,77,417,545,96
Dairy,592,2665,3855,660
Deli,43,124,216,31
Frozen_Foods,53,300,452,70
Fruits/Vegetables,541,2152,3107,538


In [41]:
persona.to_excel('persona_grocery.xlsx')

In [42]:
stats.chi2_contingency(persona)[1]

1.9980287861852168e-18

# Data Preparation

In [43]:
df_proc_inc_2014=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2014][df_labeled['items_movement']=='increase'][['Member_number','Date','itemDescription','Category',]]
df_proc_inc_2015=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2015][df_labeled['items_movement']=='increase'][['Member_number','Date','itemDescription','Category',]]
df_proc_dec_2014=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2014][df_labeled['items_movement']=='decrease'][['Member_number','Date','itemDescription','Category']]
df_proc_dec_2015=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2015][df_labeled['items_movement']=='decrease'][['Member_number','Date','itemDescription','Category']]
df_proc_rem_2014=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2014][df_labeled['items_movement']=='remain'][['Member_number','Date','itemDescription','Category']]
df_proc_rem_2015=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2015][df_labeled['items_movement']=='remain'][['Member_number','Date','itemDescription','Category']]
df_proc_new_2014=df_labeled[df_labeled['churn']=='new_cust'][df_labeled['year']==2014][['Member_number','Date','itemDescription','Category']]
df_proc_new_2015=df_labeled[df_labeled['churn']=='new_cust'][df_labeled['year']==2015][['Member_number','Date','itemDescription','Category']]

<ipython-input-43-81c65c4e4162>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_proc_inc_2014=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2014][df_labeled['items_movement']=='increase'][['Member_number','Date','itemDescription','Category',]]
<ipython-input-43-81c65c4e4162>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_proc_inc_2015=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2015][df_labeled['items_movement']=='increase'][['Member_number','Date','itemDescription','Category',]]
<ipython-input-43-81c65c4e4162>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_proc_dec_2014=df_labeled[df_labeled['churn']=='not_churn'][df_labeled['year']==2014][df_labeled['items_movement']=='decrease'][['Member_number','Date','itemDescription','Category']]
<ipython-input-43-81c65c4e4162>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame 

In [44]:
# 2014 Category
df_proc_inc_2014_cat = df_proc_inc_2014[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_inc_2014_cat = df_proc_inc_2014_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_dec_2014_cat = df_proc_dec_2014[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_dec_2014_cat = df_proc_dec_2014_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_rem_2014_cat = df_proc_rem_2014[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_rem_2014_cat = df_proc_rem_2014_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_new_2014_cat = df_proc_new_2014[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_new_2014_cat = df_proc_new_2014_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()

#2015 Category
df_proc_inc_2015_cat = df_proc_inc_2015[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_inc_2015_cat = df_proc_inc_2015_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_dec_2015_cat = df_proc_dec_2015[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_dec_2015_cat = df_proc_dec_2015_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_rem_2015_cat = df_proc_rem_2015[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_rem_2015_cat = df_proc_rem_2015_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()
df_proc_new_2015_cat = df_proc_new_2015[['Member_number','Date','Category',]].groupby(by=['Member_number','Date','Category'],as_index=False).count()
df_proc_new_2015_cat = df_proc_new_2015_cat[['Member_number','Date','Category']].groupby(by=['Member_number','Date'])['Category'].apply(lambda x: ','.join(x)).reset_index()

# 2014 Item
df_proc_inc_2014_itm = df_proc_inc_2014[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_dec_2014_itm = df_proc_dec_2014[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_rem_2014_itm = df_proc_rem_2014[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_new_2014_itm = df_proc_new_2014[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()

#2015 Item
df_proc_inc_2015_itm = df_proc_inc_2015[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_dec_2015_itm = df_proc_dec_2015[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_rem_2015_itm = df_proc_rem_2015[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()
df_proc_new_2015_itm = df_proc_new_2015[['Member_number','Date','itemDescription']].groupby(by=['Member_number','Date'])['itemDescription'].apply(lambda x: ','.join(x)).reset_index()

cat_2014=[df_proc_inc_2014_cat,df_proc_dec_2014_cat,df_proc_rem_2014_cat,df_proc_new_2014_cat]
for i in range(len(cat_2014)):
  for j in range(len(cat_2014[i])):
    cat_2014[i]['Category'][j]=cat_2014[i]['Category'][j].split(',')
  
cat_2015=[df_proc_inc_2015_cat,df_proc_dec_2015_cat,df_proc_rem_2015_cat,df_proc_new_2015_cat]
for i in range(len(cat_2015)):
  for j in range(len(cat_2015[i])):
    cat_2015[i]['Category'][j]=cat_2015[i]['Category'][j].split(',')

itm_2014=[df_proc_inc_2014_itm,df_proc_dec_2014_itm,df_proc_rem_2014_itm,df_proc_new_2014_itm]
for i in range(len(itm_2014)):
  for j in range(len(itm_2014[i])):
    itm_2014[i]['itemDescription'][j]=itm_2014[i]['itemDescription'][j].split(',')

itm_2015=[df_proc_inc_2015_itm,df_proc_dec_2015_itm,df_proc_rem_2015_itm,df_proc_new_2015_itm]
for i in range(len(itm_2015)):
  for j in range(len(itm_2015[i])):
    itm_2015[i]['itemDescription'][j]=itm_2015[i]['itemDescription'][j].split(',')

df_proc_inc_2014_fin=df_proc_inc_2014_cat.copy()
df_proc_inc_2014_fin['item']=df_proc_inc_2014_itm['itemDescription']

df_proc_inc_2015_fin=df_proc_inc_2015_cat.copy()
df_proc_inc_2015_fin['item']=df_proc_inc_2015_itm['itemDescription']

df_proc_dec_2014_fin=df_proc_dec_2014_cat.copy()
df_proc_dec_2014_fin['item']=df_proc_dec_2014_itm['itemDescription']

df_proc_dec_2015_fin=df_proc_dec_2015_cat.copy()
df_proc_dec_2015_fin['item']=df_proc_dec_2015_itm['itemDescription']

df_proc_rem_2014_fin=df_proc_rem_2014_cat.copy()
df_proc_rem_2014_fin['item']=df_proc_rem_2014_itm['itemDescription']

df_proc_rem_2015_fin=df_proc_rem_2015_cat.copy()
df_proc_rem_2015_fin['item']=df_proc_rem_2015_itm['itemDescription']

df_proc_new_2014_fin=df_proc_new_2014_cat.copy()
df_proc_new_2014_fin['item']=df_proc_new_2014_itm['itemDescription']

df_proc_new_2015_fin=df_proc_new_2015_cat.copy()
df_proc_new_2015_fin['item']=df_proc_new_2015_itm['itemDescription']

<ipython-input-44-cc4aef4b9282>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_2014[i]['Category'][j]=cat_2014[i]['Category'][j].split(',')
<ipython-input-44-cc4aef4b9282>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_2015[i]['Category'][j]=cat_2015[i]['Category'][j].split(',')
<ipython-input-44-cc4aef4b9282>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itm_2014[i]['itemDescription'][j]=itm_2014[i]['itemDescription'

In [45]:
df_proc_inc_fin_2014_cat=list(df_proc_inc_2014_fin['Category'])
df_proc_dec_fin_2014_cat=list(df_proc_dec_2014_fin['Category'])
df_proc_rem_fin_2014_cat=list(df_proc_rem_2014_fin['Category'])
df_proc_new_fin_2014_cat=list(df_proc_new_2014_fin['Category'])

df_proc_inc_fin_2015_cat=list(df_proc_inc_2015_fin['Category'])
df_proc_dec_fin_2015_cat=list(df_proc_dec_2015_fin['Category'])
df_proc_rem_fin_2015_cat=list(df_proc_rem_2015_fin['Category'])
df_proc_new_fin_2015_cat=list(df_proc_new_2015_fin['Category'])

df_proc_inc_fin_2014_itm=list(df_proc_inc_2014_fin['item'])
df_proc_dec_fin_2014_itm=list(df_proc_dec_2014_fin['item'])
df_proc_rem_fin_2014_itm=list(df_proc_rem_2014_fin['item'])
df_proc_new_fin_2014_itm=list(df_proc_new_2014_fin['item'])

df_proc_inc_fin_2015_itm=list(df_proc_inc_2015_fin['item'])
df_proc_dec_fin_2015_itm=list(df_proc_dec_2015_fin['item'])
df_proc_rem_fin_2015_itm=list(df_proc_rem_2015_fin['item'])
df_proc_new_fin_2015_itm=list(df_proc_new_2015_fin['item'])

# Data Processing

In [46]:
from apyori import apriori

In [47]:
df_cat_2014 =[df_proc_inc_fin_2014_cat,df_proc_dec_fin_2014_cat,df_proc_rem_fin_2014_cat,df_proc_new_fin_2014_cat]
rules_cat_2014=[0,1,2,3]
for i in range(len(rules_cat_2014)):
  rules_cat_2014[i] = apriori(transactions = df_cat_2014[i], min_support = 0.0000000002, min_confidence = 0.0001, min_lift = 3, min_length = 2, max_length = 10)

df_cat_2015 =[df_proc_inc_fin_2015_cat,df_proc_dec_fin_2015_cat,df_proc_rem_fin_2015_cat,df_proc_new_fin_2015_cat]
rules_cat_2015=[0,1,2,3]
for i in range(len(rules_cat_2015)):
  rules_cat_2015[i] = apriori(transactions = df_cat_2015[i], min_support = 0.0000000002, min_confidence = 0.0001, min_lift = 3, min_length = 2, max_length = 10)

df_itm_2014 =[df_proc_inc_fin_2014_itm,df_proc_dec_fin_2014_itm,df_proc_rem_fin_2014_itm,df_proc_new_fin_2014_itm]
rules_itm_2014=[0,1,2,3]
for i in range(len(rules_itm_2014)):
  rules_itm_2014[i] = apriori(transactions = df_itm_2014[i], min_support = 0.0000000002, min_confidence = 0.0001, min_lift = 3, min_length = 2, max_length = 2)

df_itm_2015 =[df_proc_inc_fin_2015_itm,df_proc_dec_fin_2015_itm,df_proc_rem_fin_2015_itm,df_proc_new_fin_2015_itm]
rules_itm_2015=[0,1,2,3]
for i in range(len(rules_itm_2015)):
  rules_itm_2015[i] = apriori(transactions = df_itm_2015[i], min_support = 0.0000000002, min_confidence = 0.0001, min_lift = 3, min_length = 2, max_length = 2)

In [48]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

In [49]:
report_inc_2014_itm = pd.DataFrame(inspect(list(rules_itm_2014[0])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_inc_2014_itm['freq']=report_inc_2014_itm['Support']*len(df_proc_inc_fin_2014_itm)
report_inc_2014_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
485,sugar,tropical fruit,0.002449,0.125000,3.293011,6.0
318,ham,whipped/sour cream,0.001633,0.166667,3.402778,4.0


In [50]:
report_inc_2015_itm = pd.DataFrame(inspect(list(rules_itm_2015[0])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_inc_2015_itm['freq']=report_inc_2015_itm['Support']*len(df_proc_inc_fin_2015_itm)
report_inc_2015_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
297,citrus fruit,specialty chocolate,0.003895,0.052448,3.014560,15.0
860,sweet spreads,tropical fruit,0.002337,0.375000,4.056531,9.0


In [51]:
report_inc_2014_cat = pd.DataFrame(inspect(list(rules_cat_2014[0])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_inc_2014_cat['freq']=report_inc_2014_cat['Support']*len(df_proc_inc_fin_2014_cat)
report_inc_2014_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
1,Pasta&Rice&Cereal,Plants,0.000816,0.074074,6.049383,2.0
23,Canned_Goods,Dairy,0.000408,0.019231,47.115385,1.0


In [52]:
report_inc_2015_cat = pd.DataFrame(inspect(list(rules_cat_2015[0])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_inc_2015_cat['freq']=report_inc_2015_cat['Support']*len(df_proc_inc_fin_2015_cat)
report_inc_2015_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
320,Condiments&Spices,Meat,0.002856,0.141026,3.291453,11.0
28,Plants,Household&Cleaning Supplies,0.002597,0.188679,3.118471,10.0


In [53]:
report_dec_2014_itm = pd.DataFrame(inspect(list(rules_itm_2014[1])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_dec_2014_itm['freq']=report_dec_2014_itm['Support']*len(df_proc_dec_fin_2014_itm)
report_dec_2014_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
91,brown bread,pet care,0.001155,0.030303,5.524721,4.0
371,frozen vegetables,pot plants,0.001155,0.040404,3.998846,4.0


In [54]:
report_dec_2015_itm = pd.DataFrame(inspect(list(rules_itm_2015[1])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_dec_2015_itm['freq']=report_dec_2014_itm['Support']*len(df_proc_dec_fin_2015_itm)
report_dec_2015_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
371,light bulbs,shopping bags,0.000620,0.333333,9.275862,1.863741
233,dessert,hygiene articles,0.001239,0.062500,6.304688,1.863741


In [55]:
report_dec_2014_cat = pd.DataFrame(inspect(list(rules_cat_2014[1])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_dec_2014_cat['freq']=report_dec_2014_itm['Support']*len(df_proc_dec_fin_2014_cat)
report_dec_2014_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
12,Health&Personal_Care,Pet_Care,0.000577,0.042553,6.408881,3.0
38,Canned_Goods,Household&Cleaning Supplies,0.000289,0.012048,41.734940,2.0


In [56]:
report_dec_2015_cat = pd.DataFrame(inspect(list(rules_cat_2015[1])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_dec_2015_cat['freq']=report_dec_2015_itm['Support']*len(df_proc_dec_fin_2015_cat)
report_dec_2015_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
172,Frozen_Foods,Bread&Bakery,0.00062,0.012346,6.641975,4.0
216,Bread&Bakery,Meat,0.00062,0.003021,4.876133,4.0


In [57]:
report_rem_2014_itm = pd.DataFrame(inspect(list(rules_itm_2014[2])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_rem_2014_itm['freq']=report_rem_2014_itm['Support']*len(df_proc_rem_fin_2014_itm)
report_rem_2014_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
296,pastry,tropical fruit,0.007353,0.151515,3.122130,5.0
239,hygiene articles,yogurt,0.004412,0.272727,3.945841,3.0


In [58]:
report_rem_2015_itm = pd.DataFrame(inspect(list(rules_itm_2015[2])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_rem_2015_itm['freq']=report_rem_2015_itm['Support']*len(df_proc_rem_fin_2015_itm)
report_rem_2015_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
286,frozen vegetables,yogurt,0.008666,0.3125,3.467548,5.0
259,finished products,whole milk,0.008666,0.6250,3.339120,5.0


In [59]:
report_rem_2014_cat = pd.DataFrame(inspect(list(rules_cat_2014[2])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_rem_2014_cat['freq']=report_rem_2014_cat['Support']*len(df_proc_rem_fin_2014_cat)
report_rem_2014_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
3,Household&Cleaning Supplies,Pasta&Rice&Cereal,0.004412,0.036585,3.554007,3.0
4,Others,Pasta&Rice&Cereal,0.002941,0.052632,5.112782,2.0


In [60]:
report_rem_2015_cat = pd.DataFrame(inspect(list(rules_cat_2015[2])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_rem_2015_cat['freq']=report_rem_2015_cat['Support']*len(df_proc_rem_fin_2015_cat)
report_rem_2015_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
18,Others,Beverages,0.008666,0.227273,3.450957,5.0
31,Bread&Bakery,Snacks,0.006932,0.033333,3.846667,4.0


In [61]:
report_new_2015_itm = pd.DataFrame(inspect(list(rules_itm_2015[3])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_new_2015_itm['freq']=report_new_2015_itm['Support']*len(df_proc_new_fin_2015_itm)
report_new_2015_itm.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
413,frozen vegetables,yogurt,0.006383,0.333333,3.601533,6.0
242,coffee,domestic eggs,0.005319,0.178571,4.094077,5.0


In [62]:
report_new_2015_cat = pd.DataFrame(inspect(list(rules_cat_2015[3])), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
report_new_2015_cat['freq']=report_new_2015_cat['Support']*len(df_proc_new_fin_2015_cat)
report_new_2015_cat.nlargest(n = 2, columns = ['freq','Lift'])

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,freq
80,Condiments&Spices,Meat,0.012766,0.166667,3.333333,12.0
86,Condiments&Spices,Meat,0.007447,0.097222,3.046296,7.0
